In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

In [ ]:
path = Path('data/Histology/100X')

In [ ]:
# Go through folder, resize anything larger than 500 pix
for folder in ['benign', 'malignant']:
    print(folder)
    verify_images(path/folder, delete=True, max_size=500)

In [ ]:
# Set up validation percent
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train='.', valid_pct=0.2,
                                 ds_tfms=get_transforms(), size=224, 
                                  num_workers=4).normalize(imagenet_stats)

In [ ]:
# Check things
print(data.classes)
data.show_batch(rows=4)

In [ ]:
## Start training
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
# Fit some data
defaults.device = torch.device('cuda')
learn.fit_one_cycle(4)

In [ ]:
# Save things
learn.save('histology_100x_v1')
learn.unfreeze()
learn.lr_find()

In [ ]:
# Plot learning rate
learn.recorder.plot()